In [1]:
import pandas as pd
import numpy as np
import requests
import re

from glob import glob
from tqdm import tqdm

import time

## Get pdb reference dataset assessment metrics.

In [2]:
def qmean_calculation(file):

    qmean_url = "https://swissmodel.expasy.org/qmean/submit/"

    token = 'a9b3d9b9a96bb9be574137b48f8d1a1a5eb4f3b8'

    result = []
    for i in tqdm(glob(file)):
        pdb_file = i

        # To upload a local file found at /path/to/my_structure.pdb
        # ('rb' is recommended to allow zip file upload)
        try: 
            response = requests.post(url=qmean_url, headers={ "Authorization": f"Token {token}" },
                                     data={ 
                                        "email": "jailann22silvasousa@gmail.com" 
                                     },
                                     files={
                                        "structure": open(i, 'rb'),
                                     })

            run = True
            while run:
                time.sleep(10)
                current_status = requests.get(response.json()["results_json"])
                status = current_status.json()['status']
                if status == 'COMPLETED':
                    result.append(current_status)
                    run = False
                 

       
            response.raise_for_status()
        
        except requests.exceptions.HTTPError as errh:
            print("Http Error:",errh)
            return result
        except requests.exceptions.ConnectionError as errc:
            print("Error Connecting:",errc)
            return result
        except requests.exceptions.Timeout as errt:
            print("Timeout Error:",errt)
            return result
        except requests.exceptions.RequestException as err:
            return result
            print("OOps: Something Else",err)

    return result

In [3]:
file = '*.pdb'

In [ ]:
results = qmean_calculation(file)

 12%|█████                                      | 11/93 [06:36<51:38, 37.79s/it]

In [ ]:
def getitem_for(d, key):
    for level in key:
        d = d[level]
    return d

def structure_data(data):
    
    qmean4_zscore_keys = ['model', 'scores', 'global_scores', 'qmean4_z_score']
    qmean4_norm_keys = ['model', 'scores', 'global_scores', 'qmean4_norm_score']
    qmeandisco_keys = ['model', 'scores', 'local_scores', 'A']
    protein_size = ['model', 'chains', 'A', 'atomseq']

    plot_data = []
    for i in range(len(results)):
        data = results[i].json()['models']

        for key, values in data.items():

            model_name = data[key]['original_name']
    
            protein_size[0] = key
            qmean4_zscore_keys[0] = key
            qmean4_norm_keys[0] = key
            qmeandisco_keys[0] = key
    

            data_structured = {
                                'model_name' : model_name,
                                'protein_size': len(getitem_for(data, protein_size)),
                                'qmean4_zscore': float(f"{getitem_for(data, qmean4_zscore_keys):.2f}"),
                                'qmean4_norm': float(f"{getitem_for(data, qmean4_norm_keys):.2}"),
                                'qmeandisco': float(f"{np.mean(np.array(getitem_for(data, qmeandisco_keys))):.2f}")
                                }
            plot_data.append(data_structured)

    return plot_data

In [ ]:
pdb_ref_metrics = structure_data(data=data)
pdb_ref = pd.DataFrame(pdb_ref_metrics)
pdb_ref.to_csv("models_qmean.csv", index=False)

## Get models assessment metrics.